## Dia 1

Neste primeiro dia, estaremos estudando os dos datasets do portal **CEAPS** e aplicar processos de tratamento e limpeza de dados nele.

Basicamente, o CEAPS contém todos os gastos que senadores brasileiros declararam, divididos por ano.

Os dados do CEAPS contêm uma série de problemas que podem dificultar a criação de análises mais aprofundadas.

Uma das primeiras coisas que faremos é identificar tais inconsistências, como campos que possuem valores nulos ou duplicados, converter campos de data que estão sendo carregados como texto, corrigir valores monetários, nomes incorretos, formatar campos de CNPJ, etc.

In [10]:
import pandas as pd

In [11]:
dados = pd.read_csv('despesa_ceaps_2022.csv', sep=';', encoding='ISO-8859-1', skiprows=1)
dados

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/22,03/01/2022,Despesa com pagamento de aluguel de imóvel par...,6000,2173614
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,26.320.603/0001-64,INFORMANAHORA,000000000000310/A,04/01/2022,Despesa com divulgação da atividade parlamenta...,1500,2173615
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,13.659.201/0001-47,LINHA PURPURA FOTO E VIDEO LTDA,107,14/01/2022,Despesa com produção de texto e edição de víde...,6000,2173616
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,23.652.846/0001-01,ROBERTO GUTIERREZ DA ROCHA M.E.I.,187,18/01/2022,Divulgação da atividade parlamentar,1000,2173618
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,08.941.827/0001-01,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,000000000001772/A,17/01/2022,Divulgação da atividade parlamentar,2000,2173617
...,...,...,...,...,...,...,...,...,...,...,...
16800,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WIXHAI,06/12/2022,"Companhia Aérea: LATAM, Localizador: WIXHAI. P...","2893,04",2191398
16801,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WITOLM,09/12/2022,"Companhia Aérea: GOL, Localizador: WITOLM. Pas...","1180,19",2192272
16802,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,THPKVQ,20/12/2022,"Companhia Aérea: TAM, Localizador: THPKVQ. Pas...","2671,9",2192274
16803,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QNN9HX,21/12/2022,"Companhia Aérea: AZUL, Localizador: QNN9HX. Pa...","1334,31",2192244


In [12]:
for i in range(2009, 2023):
    df = pd.read_csv(f'despesa_ceaps_{i}.csv', sep=';', encoding='ISO-8859-1', skiprows=1)
    dados = pd.concat([dados, df], axis=0)

dados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 332342 entries, 0 to 16804
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                332342 non-null  int64  
 1   MES                332342 non-null  int64  
 2   SENADOR            332342 non-null  object 
 3   TIPO_DESPESA       332342 non-null  object 
 4   CNPJ_CPF           331647 non-null  object 
 5   FORNECEDOR         331647 non-null  object 
 6   DOCUMENTO          309047 non-null  object 
 7   DATA               331647 non-null  object 
 8   DETALHAMENTO       189693 non-null  object 
 9   VALOR_REEMBOLSADO  332342 non-null  object 
 10  COD_DOCUMENTO      332340 non-null  float64
dtypes: float64(1), int64(2), object(8)
memory usage: 30.4+ MB


In [13]:
dados.duplicated().sum()

np.int64(16805)

#### Como é possível ver, nosso dados não possui problemas com valores duplicados

In [14]:
dados.isna().sum()

ANO                       0
MES                       0
SENADOR                   0
TIPO_DESPESA              0
CNPJ_CPF                695
FORNECEDOR              695
DOCUMENTO             23295
DATA                    695
DETALHAMENTO         142649
VALOR_REEMBOLSADO         0
COD_DOCUMENTO             2
dtype: int64

#### É perceptível que a coluna detalhamento tem muitos valores nulos, mas como ela só especifica com relação ao tipo de despesa, que neste caso não possui valores nulos, podemos simplesmente removê-la de nossa análise.

#### Com uma breve pesquisa, foi descoberto que a coluna 'DOCUMENTO' refere-se ao número da nota fiscal ou recibo apresentado pelo fornecedor. Com isso concluímos que esta coluna não nos trás muita informação, logo será removida do nosso dataframe.

#### Durante o tratamento dos dados, foram encontrados algumas inconsistências nos dados, tais qual erro em um valor da coluna 'VALOR_REEMBOLSADO' que foi facilmente resolvido e erro de data na coluna 'DATA' na qual esta tansformaremos em valor nulo e tiraremos ela de nossa análise.

#### Como a quantidade de fornecedores nulo é baixa, cosideraremos ela não sendo importante para esta análise, uma vez que ainda não foi dado qual os próximos passos do desafio.

In [15]:
dados['DATA'] = pd.to_datetime(dados['DATA'], format='%d/%m/%Y', errors='coerce')
dados['COD_DOCUMENTO'] = dados['COD_DOCUMENTO'].astype('string')
dados['CNPJ_CPF'] = dados['CNPJ_CPF'].astype('string')
dados['FORNECEDOR'] = dados['FORNECEDOR'].astype('string')
dados['SENADOR'] = dados['SENADOR'].astype('string')
dados['VALOR_REEMBOLSADO'] = [valor.replace(',','.') for valor in dados['VALOR_REEMBOLSADO']]
dados['VALOR_REEMBOLSADO'] = [valor.replace(',','.')  for valor in dados['VALOR_REEMBOLSADO']]
dados['VALOR_REEMBOLSADO'] = dados['VALOR_REEMBOLSADO'].replace('1\r\n675.55','1675.55')
dados['VALOR_REEMBOLSADO'] = dados['VALOR_REEMBOLSADO'].astype(float)
dados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 332342 entries, 0 to 16804
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ANO                332342 non-null  int64         
 1   MES                332342 non-null  int64         
 2   SENADOR            332342 non-null  string        
 3   TIPO_DESPESA       332342 non-null  object        
 4   CNPJ_CPF           331647 non-null  string        
 5   FORNECEDOR         331647 non-null  string        
 6   DOCUMENTO          309047 non-null  object        
 7   DATA               331624 non-null  datetime64[ns]
 8   DETALHAMENTO       189693 non-null  object        
 9   VALOR_REEMBOLSADO  332342 non-null  float64       
 10  COD_DOCUMENTO      332340 non-null  string        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3), string(4)
memory usage: 30.4+ MB


In [16]:
dados = dados.drop(['DETALHAMENTO', 'DOCUMENTO'], axis=1)
dados = dados.dropna()
dados = dados.reset_index()
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331624 entries, 0 to 331623
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   index              331624 non-null  int64         
 1   ANO                331624 non-null  int64         
 2   MES                331624 non-null  int64         
 3   SENADOR            331624 non-null  string        
 4   TIPO_DESPESA       331624 non-null  object        
 5   CNPJ_CPF           331624 non-null  string        
 6   FORNECEDOR         331624 non-null  string        
 7   DATA               331624 non-null  datetime64[ns]
 8   VALOR_REEMBOLSADO  331624 non-null  float64       
 9   COD_DOCUMENTO      331624 non-null  string        
dtypes: datetime64[ns](1), float64(1), int64(3), object(1), string(4)
memory usage: 25.3+ MB
